# 2D EM Simulation

In [54]:
import os
import sys
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import emsimsirenpinn.SIREN_PINN as SP
import torch
import numpy as np
import matplotlib.pyplot as plt

In [55]:
# import importlib
# importlib.reload(SP)

In [56]:
torch.manual_seed(42)
np.random.seed(42)


In [57]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




n1 = 200  #nx
n2 = 200  #nz
d1 = 0.01
d2 = 0.01
npml = 20

# f = 0.5  # Frequency  # 0.015-0.15 -> omega 0.1 - 1
# f = 1
f = 5
PI = 3.1415926
omega = 2.0 * PI * f # Angular frequency

print(omega)

sigma=3.0

niter = 100000 #Adam iterations
lr = 1e-4 # Adam learning rate
lbfgs_max_iter = 30000 # LBFGS iterations


n1_s = 40 # Source locations
n2_s = 40

perm_model = SP.generate_permittivity_model(n1, n2)
pmla, pmlb, pmlc = SP.generate_boundary_coeff(n1, n2, npml, f, f)
point_source = SP.generate_point_source(n1, 
                                        n2, 
                                        n1_s, 
                                        n2_s, 
                                        sigma, 
                                        max_amplitude=100.0, 
                                        apply_threshold=True)
grid_n1, grid_n2 = SP.generate_mesh_grid(n1, n2, d1, d2)

da_dx, da_dy = torch.gradient(torch.tensor(pmla), spacing=(d1, d2))
db_dx, db_dy = torch.gradient(torch.tensor(pmlb), spacing=(d1, d2))
da_dx = da_dx.numpy()
da_dy = da_dy.numpy()
db_dx = db_dx.numpy()
db_dy = db_dy.numpy()

m = SP.prepare_data_training(perm_model)
A = SP.prepare_data_training(pmla)
B = SP.prepare_data_training(pmlb)
C = SP.prepare_data_training(pmlc)
ps = SP.prepare_data_training(point_source)
g1 = SP.prepare_data_training(grid_n1)
g2 = SP.prepare_data_training(grid_n2)

da_dx_p = SP.prepare_data_training(da_dx)
da_dy_p = SP.prepare_data_training(da_dy)
db_dx_p = SP.prepare_data_training(db_dx)
db_dy_p = SP.prepare_data_training(db_dy)

print(da_dx_p.shape, da_dy_p.shape, db_dx_p.shape, db_dy_p.shape)


N = m.shape[0]
idx = np.random.choice(N, int(N), replace=False)


pinn_model = SP.PINN(
    g1[idx, :], g2[idx, :],
    A[idx, :], B[idx, :], C[idx, :],
    ps[idx, :], m[idx, :],
    omega, da_dx_p[idx, :], db_dy_p[idx, :], lr,
    niter,
    lbfgs_max_iter = lbfgs_max_iter,
    in_features=2, 
    hidden_features=40, 
    hidden_layers=8, 
    out_features=2, 
    w0=1.0, 
    w0_initial=30.0,  # SIREN specific
    device=device,
).to(device)

print("Start training")
loss_all = pinn_model.train_model()
print("End training")

print("Prediction")
# Prediction
u_real_pred, u_imag_pred = pinn_model.predict(g1, g2)



In [58]:
grid_n1.shape, grid_n2.shape, perm_model.shape, point_source.shape, pmla.shape, pmlb.shape, pmlc.shape, m.shape, A.shape, B.shape, C.shape, da_dx_p.shape, da_dy_p.shape, db_dx_p.shape, db_dy_p.shape

In [59]:
for name, param in pinn_model.model.named_parameters():
    if param.requires_grad:  # Check if the parameter is trainable
        print(f"Layer: {name}, Shape: {param.shape}")
        print(f"Weights/Biases:\n{param.data}")

In [60]:
plt.imshow(pmla.real.T)
plt.colorbar()
plt.xlabel("x")
plt.ylabel("z")
plt.title("PML Coefficient A - REAL")
plt.show()


plt.imshow(pmla.imag.T)
plt.colorbar()
plt.xlabel("x")
plt.ylabel("z")
plt.title("PML Coefficient A - IMAG")
plt.show()



plt.imshow(pmlb.real.T)
plt.colorbar()
plt.xlabel("x")
plt.ylabel("z")
plt.title("PML Coefficient B - REAL")
plt.show()


plt.imshow(pmlb.imag.T)
plt.colorbar()
plt.xlabel("x")
plt.ylabel("z")
plt.title("PML Coefficient B - IMAG")
plt.show()


plt.imshow(pmlc.real.T)
plt.colorbar()
plt.xlabel("x")
plt.ylabel("z")
plt.title("PML Coefficient C - REAL")
plt.show()


plt.imshow(pmlc.imag.T)
plt.colorbar()
plt.xlabel("x")
plt.ylabel("z")
plt.title("PML Coefficient C - IMAG")
plt.show()




In [61]:

plt.imshow(point_source.T)
plt.title("Point Source")
plt.colorbar()
plt.xlabel("x")
plt.ylabel("z")
plt.show()

plt.imshow(perm_model.T)
plt.title("Permittivity Model")
plt.colorbar()
plt.xlabel("x")
plt.ylabel("z")
plt.show()

In [62]:
plt.plot(loss_all)
plt.yscale('log')

In [63]:
plt.figure(figsize=(8, 8))

vmin = -1
vmax = -vmin

# cmap = 'viridis's
cmap = 'gray'

plt.subplot(2, 1, 1)
plt.imshow(u_real_pred.reshape([n1, n2]).T, aspect='equal', cmap=cmap, vmin=vmin, vmax=vmax)
# plt.gca().set_aspect('equal')
plt.colorbar(label='u(x,y)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Predicted-Real')

plt.subplot(2, 1, 2)
plt.imshow(u_imag_pred.reshape([n1, n2]).T, aspect='equal', cmap=cmap, vmin=vmin, vmax=vmax)
# plt.gca().set_aspect('equal')  
plt.colorbar(label='u(x,y)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Predicted-Imaginary')

plt.tight_layout()
plt.show()